In [1]:
import pandas as pd
import numpy as np
import common

from pyecharts.charts import Grid, Kline, Line, Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType

import talib.abstract as ta

获取数据

In [2]:
file = '../freqtrade/user_data/data/binance/futures/BTC_USDT_USDT-6h-futures.json'

In [3]:
df = common.read_kline_data(file)

添加MA和EMA

In [4]:
df['ma20'] = ta.MA(df, timeperiod=20)
df['ma60'] = ta.MA(df, timeperiod=60)
df['ma120'] = ta.MA(df, timeperiod=120)

df['ema20'] = ta.EMA(df, timeperiod=20)
df['ema60'] = ta.EMA(df, timeperiod=60)
df['ema120'] = ta.EMA(df, timeperiod=120)

计算密集程度

In [5]:
df['intensity_ma'] = (abs(df['ma20'] - df['ma60']) + abs(df['ma60'] - df['ma120']) + abs(df['ma120'] - df['ma20'])) /\
                    ((df['ma20'] + df['ma60'] + df['ma120']) / 3)

In [6]:
df = df.dropna()

### 构建图表

初始化K线数据

In [7]:
data = []
for _, row in df.iterrows():
    data.append([
        row["open"], 
        row["close"], 
        row["low"], 
        row["high"], 
    ])

开始构建K线

构建K线的指标MA和EMA

In [8]:
kline = common.create_kline_chart(dataframe=df)

In [9]:
line = common.create_line_chart(df, ['ma20', 'ma60', 'ma120'])

成交量

In [10]:
bar = common.create_bar_chart(df, ['volume'])

### 合并

In [11]:
overlap_kline_line = kline.overlap(line)
grid_chart = Grid(init_opts=opts.InitOpts(width='100%', height='600px'))
grid_chart.add(
    overlap_kline_line,
    grid_opts=opts.GridOpts(pos_left="5%", pos_right="5%", height="60%"),
)

grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(
        pos_left="5%", pos_right="5%", pos_top="78%", height="16%"
    ),
)

In [12]:
grid_chart.render_notebook()